<a href="https://colab.research.google.com/github/Road2SKA/python_hpc_tutorial/blob/main/mpi4py_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distributed Parallel Programming Patterns using mpi4py

The message passing interface (MPI) is a bottom-up approach to parallel programming. Message passing can be used on a single multicore computer or with a cluster of computers. The implementation of the message passing functions are based on original work by Joel Adams:

Adams, Joel C. "Patternlets: A Teaching Tool for Introducing Students to Parallel Design Patterns." 2015 IEEE International Parallel and Distributed Processing Symposium Workshop. IEEE, 2015.

To run these examples, first you will need to install the mpi4py library by running this code (this will usually take a while to install the first time):

In [1]:
! pip install mpi4py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.4 MB/s eta 0:00:00


**Important:** you will have to re-run this cell after you you get disconnected for a fairly long time.

## Simple example

This code forms the basis of all of the other examples that follow. It is the fundamental way we structure parallel programs today.


In [2]:
%%writefile my_program.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    print("Greetings from process {} of {} on {}"\
    .format(id, numProcesses, myHostName))

########## Run the main function
main()


Writing my_program.py


Let's examine the variables created in lines 5-8 carefully.

1. *comm* The fundamental notion with this type of computing is a *process* running independently on the computer. With one single program like this, we can specify that we want to start several processes, each of which can **communicate**. The mechanism for communication is initialized when the program starts up, and the object that represents the means of using communication between processes is called MPI.COMM_WORLD, which we place in the variable comm.

2. *id* Every process can identify itself with a number. We get that number by asking *comm* for it using Get_rank().

3. *numProcesses* It is helpful to know haw many processes have started up, because this can be specified differently every time you run this type of program. Asking *comm* for it is done with Get_size().

4. *myHostName* When you run this code on a cluster of computers, it is sometimes useful to know which computer is running a certain piece of code. A particular computer is often called a 'host', which is why we call this variable myHostName, and get it by asking *comm* to provide it with Get_processor_name().

These four variables are often used in every MPI program. The first three are often needed for writing correct programs, and the fourth one is often used for debugging and analysis of where certain computations are running.

Next we see how we can use the mpirun program to execute the above python code using 4 processes. The value after -np is the number of processes to use when running the file of python code saved when executing the previous code cell.

In [3]:
! mpirun --allow-run-as-root --oversubscribe -np 8 python my_program.py

Greetings from process 2 of 8 on 9cf7bbd8e3da
Greetings from process 6 of 8 on 9cf7bbd8e3da
Greetings from process 5 of 8 on 9cf7bbd8e3da
Greetings from process 0 of 8 on 9cf7bbd8e3da
Greetings from process 4 of 8 on 9cf7bbd8e3da
Greetings from process 1 of 8 on 9cf7bbd8e3da
Greetings from process 3 of 8 on 9cf7bbd8e3da
Greetings from process 7 of 8 on 9cf7bbd8e3da


The fundamental idea of message passing programs can be illustrated like this:

![picture](https://drive.google.com/uc?id=1wpQaFiaubIcQBV9Lw_jwOU0y2-K-EChW)

Each process is set up within a communication network to be able to communicate with every other process via communication links. Each process is set up to have its own number, or id, which starts at 0.

**Note:** Each process holds its own copies of the above 4 data variables. **So even though there is one single program, it is running multiple times in separate processes, each holding its own data values.** The print line at the end of main() represents the multiple different data output produced by each process.


## Prime numbers
Let's try to implement the prime number function using MPI. Remember that our default serial implementation is:



In [4]:
import math
def is_prime(n):
    if n <= 1:
        return False
    n_sqrt = math.floor(math.sqrt(n)) + 1
    for i in range(2, n_sqrt):
        if n % i == 0:
            return False
    return True
n = 6783858998837822
%time res=is_prime(n)
print("{0} prime: {1}".format(n,res))

CPU times: user 10 µs, sys: 1 µs, total: 11 µs
Wall time: 15.7 µs
6783858998837822 prime: False


Let's try to implement this using MPI:

In [5]:
%%writefile mpi_primes.py
from mpi4py import MPI
import numpy as np
import math
def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code


    print("Greetings task {} of {} on {}".format(id, numProcesses, myHostName))

    # here is our input. We could also read this from the command line instead of hard-coding the value
    n = 6783858998837822

    is_prime = True # our default guess.
    if n <= 1:
      is_prime = False
    else:
      n_sqrt = math.floor(math.sqrt(n)) + 1
      # split up this loop over the different processes.
      check_nums = np.split(np.arange(2,n_sqrt),numProcesses)
      check_nums = check_nums[id]

      for i in check_nums:
        if n % i == 0:
            is_prime = False
    print("task {} finds {}".format(id, is_prime))

########## Run the main function
main()


Writing mpi_primes.py


In [6]:
! mpirun --allow-run-as-root --oversubscribe -np 7 python mpi_primes.py

Greetings task 4 of 7 on 9cf7bbd8e3da
Greetings task 2 of 7 on 9cf7bbd8e3da
Greetings task 3 of 7 on 9cf7bbd8e3da
Greetings task 5 of 7 on 9cf7bbd8e3da
Greetings task 6 of 7 on 9cf7bbd8e3da
Greetings task 1 of 7 on 9cf7bbd8e3da
Greetings task 0 of 7 on 9cf7bbd8e3da
task 3 finds True
task 5 finds True
task 0 finds False
task 1 finds True
task 4 finds True
task 2 finds True
task 6 finds True


Let's try to modify the code so that we only get 1 result instead of 7. We'll use one process, process 0, as our special "conductor" process which collects all of the results of the others. To do this we will need to use communication, the "message passing" of MPI.

There are many different ways to pass mssages:
point-to-point, broadcasting, scatter/gather, etc. You can find information on all of the different patterns at [https://mpi4py.readthedocs.io](https://mpi4py.readthedocs.io).

Here, we will

In [17]:
%%writefile mpi_primes2.py
from mpi4py import MPI
import numpy as np
import math
def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code


    print("Greetings task {} of {} on {}".format(id, numProcesses, myHostName))

    # here is our input. We could also read this from the command line instead of hard-coding the value
    n = 6783858998837822

    is_prime = True # our default guess.
    if n <= 1:
      is_prime = False

    # task 0 will define the loop and send it to the other processes using SCATTER
    if id == 0:
      n_sqrt = math.floor(math.sqrt(n)) + 1
      # split up this loop over the different processes.
      check_nums = np.split(np.arange(2,n_sqrt),numProcesses)
    else:
      check_nums = None
    check_nums = comm.scatter(check_nums, root=0)

    print("task {} received {}".format(id, check_nums))

    # now each task checks its part of the list
    for i in check_nums:
        if n % i == 0:
            is_prime = False

    # task 0 will collect all of the work done by other processes using GATHER
    result = comm.gather(is_prime, root=0)
    if id == 0:
      print("task {} received {}".format(id, result))
      is_prime = not (False in result)
      print("{} prime: {}".format(n, is_prime))

    else:
      assert result is None


########## Run the main function
main()

Overwriting mpi_primes2.py


In [18]:
! mpirun --allow-run-as-root --oversubscribe -np 7 python mpi_primes2.py

Greetings task 2 of 7 on 9cf7bbd8e3da
Greetings task 3 of 7 on 9cf7bbd8e3da
Greetings task 4 of 7 on 9cf7bbd8e3da
Greetings task 1 of 7 on 9cf7bbd8e3da
Greetings task 6 of 7 on 9cf7bbd8e3da
Greetings task 5 of 7 on 9cf7bbd8e3da
Greetings task 0 of 7 on 9cf7bbd8e3da
task 1 received [11766314 11766315 11766316 ... 23532623 23532624 23532625]
task 2 received [23532626 23532627 23532628 ... 35298935 35298936 35298937]
task 3 received [35298938 35298939 35298940 ... 47065247 47065248 47065249]
task 5 received [58831562 58831563 58831564 ... 70597871 70597872 70597873]
task 4 received [47065250 47065251 47065252 ... 58831559 58831560 58831561]
task 0 received [       2        3        4 ... 11766311 11766312 11766313]
task 6 received [70597874 70597875 70597876 ... 82364183 82364184 82364185]
task 0 received [False, True, True, True, True, True, True]
6783858998837822 prime: False


Setting up the different tasks and communication between the different processes means that MPI will be slower than single-process execution with vectorization. You will see the best performance when running across many nodes on large datasets.

**Exercise:** rerun the prime-funder script, using varying numbers of processes from 1 through 8 (i.e., vary the argument after -np). Explain what stays the same and what changes as the number of processes changes.